Using Amazon's sales activity along with GDP --- Ensemble Model

In [1]:
%%capture
!pip install pycaret[full]

In [2]:
import pandas as pd
import numpy as np 
from pycaret.regression import *

In [3]:
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv',index_col='row_id')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv',index_col='row_id')

GDP Dataset Year wise for Finaland, Norway and Sweden

In [4]:
gdp_df = pd.read_csv('../input/gdp-data/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')

In [5]:
amazon = pd.read_csv('../input/amazonn/multiTimeline (2).csv')
temp = amazon.index.tolist()
amzn = []
for i in range(1,len(temp)):
    amzn.append([temp[i][0],temp[i][1],temp[i][2],amazon.iloc[i,0]])

Creating Monthly sales Activity country ,( Month, Year ) wise and saving it in hashmap.. Working on Daily sales Activity currently using Amazon's sales activity only

In [6]:
swed = {}
finl = {}
norw = {}

In [7]:
for i in range(len(amzn)):
    if (amzn[i][0][:4],amzn[i][0][5:7]) not in swed:
        swed[(int(amzn[i][0][5:7]),int(amzn[i][0][:4]))] = amzn[i][1]
        finl[(int(amzn[i][0][5:7]),int(amzn[i][0][:4]))] = amzn[i][2]
        norw[(int(amzn[i][0][5:7]),int(amzn[i][0][:4]))] = amzn[i][3]
    

In [8]:
finl

{(1, 2015): '54',
 (2, 2015): '46',
 (3, 2015): '47',
 (4, 2015): '55',
 (5, 2015): '39',
 (6, 2015): '44',
 (7, 2015): '54',
 (8, 2015): '53',
 (9, 2015): '40',
 (10, 2015): '50',
 (11, 2015): '53',
 (12, 2015): '52',
 (1, 2016): '53',
 (2, 2016): '39',
 (3, 2016): '48',
 (4, 2016): '49',
 (5, 2016): '39',
 (6, 2016): '39',
 (7, 2016): '51',
 (8, 2016): '47',
 (9, 2016): '48',
 (10, 2016): '50',
 (11, 2016): '67',
 (12, 2016): '56',
 (1, 2017): '54',
 (2, 2017): '45',
 (3, 2017): '48',
 (4, 2017): '53',
 (5, 2017): '47',
 (6, 2017): '52',
 (7, 2017): '60',
 (8, 2017): '54',
 (9, 2017): '46',
 (10, 2017): '54',
 (11, 2017): '64',
 (12, 2017): '49',
 (1, 2018): '57',
 (2, 2018): '53',
 (3, 2018): '64',
 (4, 2018): '42',
 (5, 2018): '45',
 (6, 2018): '43',
 (7, 2018): '48',
 (8, 2018): '53',
 (9, 2018): '49',
 (10, 2018): '62',
 (11, 2018): '62',
 (12, 2018): '49',
 (1, 2019): '51',
 (2, 2019): '54',
 (3, 2019): '44',
 (4, 2019): '43',
 (5, 2019): '42',
 (6, 2019): '51',
 (7, 2019): '50'

Based on amazon's sales activity during that period adding it to our test and train Data

In [9]:
gdp_df.columns = ['year', 'Finland', 'Norway', 'Sweden']

gdp_df = pd.melt(gdp_df, id_vars=['year'], value_vars=['Finland', 'Norway', 'Sweden'], var_name='country', value_name='gdp')
gdp_df['year'] = 'Y' + gdp_df['year'].astype(str)

In [10]:
def pre_process(df):
    
    df['date'] = pd.to_datetime(df['date'])
    df['week']= df['date'].dt.week
    df['month'] = df['date'].dt.month
    df['year'] = 'Y'+df['date'].dt.year.astype(str)
    df['quarter'] = 'Q'+df['date'].dt.quarter.astype(str)
    df['day'] = df['date'].dt.day
    df['dayofyear'] = df['date'].dt.dayofyear
    df.loc[(df.date.dt.is_leap_year) & (df.dayofyear >= 60),'dayofyear'] -= 1
    df['weekend'] = df['date'].dt.weekday >=5
    df['weekday'] = 'WD' + df['date'].dt.weekday.astype(str)
    #df['Newyear'] = 0
    #df.loc[(df.date.dt.dayofyear >= 300 ) | (df.date.dt.dayofyear <30)]['Newear'] = 1
    #df['Newyearb'] = df['dayofyear'] > 300
    #df['Newyeara'] = df['dayofyear'] < 30
    df.drop(columns=['date'],inplace=True)

pre_process(train)
pre_process(test)

#combining GDP Dataset
train = train.merge(gdp_df, how='left', on=['year', 'country'])
test = test.merge(gdp_df, how='left', on=['year', 'country'])

#initialize new column for amazon data
train['amazon'] = 0
test['amazon'] = 0

for i in range(len(train)):
    if train.iloc[i,0] == 'Sweden':
        train.amazon.iloc[i] = int(swed[(int(train.iloc[i]['month']),int(train.iloc[i]['year'][1:]))]) 
    elif train.iloc[i,0] == 'Finland':
        train.amazon.iloc[i] = int(finl[(int(train.iloc[i]['month']),int(train.iloc[i]['year'][1:]))]) 
    elif train.iloc[i,0] == 'Norway':
        train.amazon.iloc[i] = int(norw[(int(train.iloc[i]['month']),int(train.iloc[i]['year'][1:]))]) 

        
        
for i in range(len(test)):
    if test.iloc[i,0] == 'Sweden':
        test.amazon.iloc[i] = int(swed[(int(test.iloc[i]['month']),int(test.iloc[i]['year'][1:]))])
    elif test.iloc[i,0] == 'Finland':
        test.amazon.iloc[i] = int(finl[(int(test.iloc[i]['month']),int(test.iloc[i]['year'][1:]))])
    elif test.iloc[i,0] == 'Norway':
        test.amazon.iloc[i] = int(norw[(int(test.iloc[i]['month']),int(test.iloc[i]['year'][1:]))]) 


In [11]:
train['amazon'] = train['amazon'].astype('int') 
test['amazon'] = test['amazon'].astype('int') 
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26298 entries, 0 to 26297
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    26298 non-null  object 
 1   store      26298 non-null  object 
 2   product    26298 non-null  object 
 3   num_sold   26298 non-null  int64  
 4   week       26298 non-null  int64  
 5   month      26298 non-null  int64  
 6   year       26298 non-null  object 
 7   quarter    26298 non-null  object 
 8   day        26298 non-null  int64  
 9   dayofyear  26298 non-null  int64  
 10  weekend    26298 non-null  bool   
 11  weekday    26298 non-null  object 
 12  gdp        26298 non-null  float64
 13  amazon     26298 non-null  int64  
dtypes: bool(1), float64(1), int64(6), object(6)
memory usage: 2.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6570 entries, 0 to 6569
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -

(None, None)

In [12]:
train.head()

,country,store,product,num_sold,week,month,year,quarter,day,dayofyear,weekend,weekday,gdp,amazon
0,Finland,KaggleMart,Kaggle Mug,329,1,1,Y2015,Q1,1,1,False,WD3,234.44,54
1,Finland,KaggleMart,Kaggle Hat,520,1,1,Y2015,Q1,1,1,False,WD3,234.44,54
2,Finland,KaggleMart,Kaggle Sticker,146,1,1,Y2015,Q1,1,1,False,WD3,234.44,54
3,Finland,KaggleRama,Kaggle Mug,572,1,1,Y2015,Q1,1,1,False,WD3,234.44,54
4,Finland,KaggleRama,Kaggle Hat,911,1,1,Y2015,Q1,1,1,False,WD3,234.44,54


In [13]:
# Credit to https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
def SMAPE(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 200 * np.mean(diff)

In [14]:
reg = setup(data = train,
            target = 'num_sold',
      transform_target=True,
      transform_target_method='box-cox',  # https://www.statisticshowto.com/box-cox-transformation/
      silent=True,
      feature_interaction = True,
      normalize=True,
      normalize_method='robust',
      session_id = 42,  # It is equivalent to 'random_state' in scikit-learn.
      data_split_shuffle = False,  # When set to False, prevents shuffling of rows during 'train_test_split'.
      fold=10)

,Description,Value
0,session_id,42
1,Target,num_sold
2,Original Data,"(26298, 14)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,8
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(18408, 66)"


In [15]:
add_metric('SMAPE', 'SMAPE', SMAPE, greater_is_better = False)
top =compare_models(sort = 'SMAPE',n_select = 2, include = ['catboost','lightgbm','gbr','xgboost'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE,TT (Sec)
catboost,CatBoost Regressor,25.4831,2453.3197,45.3749,0.9665,0.0874,0.0635,6.3798,7.3570
lightgbm,Light Gradient Boosting Machine,25.2441,2070.8036,42.8242,0.9703,0.0857,0.0637,6.4005,0.3400
xgboost,Extreme Gradient Boosting,27.6227,2519.9256,47.6831,0.9633,0.0925,0.0691,6.9239,32.3730
gbr,Gradient Boosting Regressor,28.7307,2486.9274,47.7189,0.9632,0.0958,0.0732,7.3357,2.3720


In [16]:
blend = blend_models(top)
predict_model(blend)

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
0,29.8007,3092.1402,55.6070,0.9613,0.0994,0.0680,6.9524
1,29.7274,2381.4928,48.8005,0.9656,0.1010,0.0805,7.7874
2,14.9455,477.4580,21.8508,0.9850,0.0594,0.0487,4.7368
3,24.3433,1666.1732,40.8188,0.9754,0.0860,0.0630,6.0969
4,34.2529,4022.5852,63.4239,0.9515,0.1132,0.0782,8.0725
5,19.5001,968.8095,31.1257,0.9779,0.0728,0.0588,5.7226
6,15.2907,491.6550,22.1733,0.9845,0.0611,0.0470,4.7516
7,30.2875,3847.3527,62.0270,0.9567,0.0890,0.0594,6.1974
8,35.0858,4226.6079,65.0124,0.9522,0.1121,0.0763,7.9448
9,15.2564,491.0158,22.1589,0.9872,0.0545,0.0436,4.4001


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
0,Voting Regressor,41.5469,5273.2163,72.6169,0.9378,0.1149,0.0854,9.0808


,week,day,dayofyear,gdp,amazon,country_Finland,country_Norway,country_Sweden,store_KaggleMart,product_Kaggle Hat,...,quarter_Q2_multiply_product_Kaggle Hat,day_multiply_week,store_KaggleMart_multiply_country_Norway,gdp_multiply_week,quarter_Q1_multiply_product_Kaggle Hat,product_Kaggle Sticker_multiply_store_KaggleMart,amazon_multiply_day,product_Kaggle Hat_multiply_country_Norway,num_sold,Label
0,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,1.0,0.0,...,0.0,0.141667,0.0,0.439628,0.0,0.0,0.266667,0.0,223,223.048725
1,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,1.0,1.0,...,0.0,0.141667,0.0,0.439628,0.0,0.0,0.266667,0.0,303,304.819680
2,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,1.0,0.0,...,0.0,0.141667,0.0,0.439628,0.0,1.0,0.266667,0.0,112,108.202268
3,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,0.0,0.0,...,0.0,0.141667,0.0,0.439628,0.0,0.0,0.266667,0.0,419,388.886464
4,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,0.0,1.0,...,0.0,0.141667,0.0,0.439628,0.0,0.0,0.266667,0.0,550,532.629112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7885,-1.000000,1.0,1.134503,0.678376,0.250000,0.0,0.0,1.0,1.0,1.0,...,0.0,-1.000000,0.0,-0.678376,0.0,0.0,0.250000,0.0,823,670.675428
7886,-1.000000,1.0,1.134503,0.678376,0.250000,0.0,0.0,1.0,1.0,0.0,...,0.0,-1.000000,0.0,-0.678376,0.0,1.0,0.250000,0.0,250,205.919284
7887,-1.000000,1.0,1.134503,0.678376,0.250000,0.0,0.0,1.0,0.0,0.0,...,0.0,-1.000000,0.0,-0.678376,0.0,0.0,0.250000,0.0,1004,600.706008
7888,-1.000000,1.0,1.134503,0.678376,0.250000,0.0,0.0,1.0,0.0,1.0,...,0.0,-1.000000,0.0,-0.678376,0.0,0.0,0.250000,0.0,1441,1127.818390


In [17]:
final_blend = finalize_model(blend)
predict_model(final_blend)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
0,Voting Regressor,16.7699,647.7807,25.4515,0.9924,0.0505,0.0402,4.0218


,week,day,dayofyear,gdp,amazon,country_Finland,country_Norway,country_Sweden,store_KaggleMart,product_Kaggle Hat,...,quarter_Q2_multiply_product_Kaggle Hat,day_multiply_week,store_KaggleMart_multiply_country_Norway,gdp_multiply_week,quarter_Q1_multiply_product_Kaggle Hat,product_Kaggle Sticker_multiply_store_KaggleMart,amazon_multiply_day,product_Kaggle Hat_multiply_country_Norway,num_sold,Label
0,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,1.0,0.0,...,0.0,0.141667,0.0,0.439628,0.0,0.0,0.266667,0.0,223,226.008562
1,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,1.0,1.0,...,0.0,0.141667,0.0,0.439628,0.0,0.0,0.266667,0.0,303,303.153316
2,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,1.0,0.0,...,0.0,0.141667,0.0,0.439628,0.0,1.0,0.266667,0.0,112,110.020674
3,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,0.0,0.0,...,0.0,0.141667,0.0,0.439628,0.0,0.0,0.266667,0.0,419,393.004594
4,0.708333,0.2,0.707602,0.620652,1.333333,0.0,0.0,1.0,0.0,1.0,...,0.0,0.141667,0.0,0.439628,0.0,0.0,0.266667,0.0,550,529.022704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7885,-1.000000,1.0,1.134503,0.678376,0.250000,0.0,0.0,1.0,1.0,1.0,...,0.0,-1.000000,0.0,-0.678376,0.0,0.0,0.250000,0.0,823,861.045729
7886,-1.000000,1.0,1.134503,0.678376,0.250000,0.0,0.0,1.0,1.0,0.0,...,0.0,-1.000000,0.0,-0.678376,0.0,1.0,0.250000,0.0,250,260.837456
7887,-1.000000,1.0,1.134503,0.678376,0.250000,0.0,0.0,1.0,0.0,0.0,...,0.0,-1.000000,0.0,-0.678376,0.0,0.0,0.250000,0.0,1004,899.529510
7888,-1.000000,1.0,1.134503,0.678376,0.250000,0.0,0.0,1.0,0.0,1.0,...,0.0,-1.000000,0.0,-0.678376,0.0,0.0,0.250000,0.0,1441,1452.121852


In [18]:
preds = predict_model(final_blend, data=test)
sub = pd.DataFrame(list(zip(test.index,preds.Label)),columns = ['row_id', 'num_sold'])
for i in range(len(sub)):
    sub.iloc[i,0] = i+26298
sub.to_csv('submission.csv', index = False)
print(sub.head(),sub.describe())

   row_id    num_sold
0   26298  404.566707
1   26299  586.144659
2   26300  174.313684
3   26301  680.648643
4   26302  998.464565              row_id     num_sold
count   6570.000000  6570.000000
mean   29582.500000   403.553325
std     1896.739966   264.945065
min    26298.000000   100.710881
25%    27940.250000   200.027559
50%    29582.500000   333.048909
75%    31224.750000   529.669559
max    32867.000000  2777.292218
